In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey="bronze", key="bronze_output", debugValue={})
silver_data = dbutils.jobs.taskValues.get(taskKey="silver", key="silver_output", debugValue={})
start_date = bronze_output.get("start_date", "")
silver_path = bronze_output.get("silver_path", "")
gold_path = bronze_output.get("gold_path", "")
print(f"Start Date: {start_date}, gold path: {gold_path}")

In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
df=spark.read.parquet(silver_data).filter(col('time')>start_date)

In [0]:
df=df.limit(10)


In [0]:
def get_country_code(lat,lon):
    try:
        coordinates=(float(lat),float(lon))
        result= rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates:{coordinates}->{result}")
        return result
    except Exception as e:
        print(f"Error in processing coordinates:{lat},{lon}->{str(e)}")
        return None

In [0]:
get_country_code_udf=udf(get_country_code,StringType())

In [0]:
get_country_code(48.858849,2.29435)

In [0]:
df_with_location=\
    df.\
    withColumn('country_code',get_country_code_udf(col('latitude'),col('longitude')))

In [0]:
df_with_location_sig_class=\
    df_with_location.\
    withColumn('sig_class',
                when(col('sig')<100,"low").\
                when((col('sig')>=100)&(col('sig')<500),"moderate").\
                otherwise("high")
                )


In [0]:
gold_output_path=f"{gold_path}earthquake_events_gold/"

In [0]:
df_with_location_sig_class.write.mode("append").parquet(gold_output_path)